In [ ]:
#easy Connect 4
import random
import numpy as np
import pygame
import sys
import math

ROW_COUNT = 6
COLUMN_COUNT = 7
SQSIZE = 130
RADIUS = int(SQSIZE/2 -5)
BLUE = (0,100,255)
#BLUE = (212,175,55)

GREY = (30,30,30)
YELLOW = (255,100,0)
#YELLOW = (255,0,0)
BLACK = (0,0,0)
GREEN = (30,255,20)


PLAYER =0
AI =1

WINDOW_LENGTH = 4

EMPTY =0
PLAYER_PIECE =1
AI_PIECE =2

def create_board():
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))
    return board

def drop_piece(board,row,col,piece):
    board[row][col] = piece  
    
def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0;

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] ==0 :
            return r
    
def print_board(board):
    print(np.flip(board,0))
    
def winning_move(board, p):
    
    #checking horizontal
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if(board[r][c] == p and board[r][c+1]==p and board[r][c+2]==p and board[r][c+3]==p):
                return True
    
    #Check vertical win
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT):
            if(board[r][c] == p and board[r+1][c]==p and board[r+2][c]==p and board[r+3][c]==p):
                return True
            
    #check positive slope win
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            if(board[r][c]==p and board[r+1][c+1]==p and board[r+2][c+2]==p and board[r+3][c+3]==p):
                return True
    
    #check negative slope win
    for r in range(ROW_COUNT-3):
        for c in range (3,COLUMN_COUNT):
            if(board[r][c]==p and board[r+1][c-1]==p and board[r+2][c-2] and board[r+3][c-3]==p):
                return True
            
#board with graphics :)
def draw_board(board,screen):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, BLUE, (c*SQSIZE, r*SQSIZE+SQSIZE, SQSIZE, SQSIZE))
            pygame.draw.circle(screen, BLACK, (int(c*SQSIZE+SQSIZE/2), int(r*SQSIZE+SQSIZE+SQSIZE/2)), RADIUS)
        
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == 1:
                pygame.draw.circle(screen, GREEN, (int(c*SQSIZE+SQSIZE/2), height-int(r*SQSIZE+SQSIZE/2)), RADIUS)
            elif board[r][c] == 2: 
                pygame.draw.circle(screen, YELLOW, (int(c*SQSIZE+SQSIZE/2), height-int(r*SQSIZE+SQSIZE/2)), RADIUS)
    pygame.display.update()


    
def evaluate_window(window, piece):
    score =0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE
        
    if window.count(piece) ==4:
        score += 100
    elif window.count(piece) ==3 and window.count(EMPTY)==1:
        score += 5
    elif window.count(piece) ==2 and window.count(EMPTY) ==2:
        score += 2
    
    if window.count(opp_piece) ==3 and window.count(EMPTY) ==1:
        score -= 4
    
    return score

def score_position(board,piece):
    score =0
    
    #Score center column
    center_array = [int(i) for i in list(board[:,COLUMN_COUNT//2])]
    center_count = center_array.count(piece)
    score += center_count * 3
    
        
    ##Score Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT-3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window,piece)
            
    ##Score verticle
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            window  = col_array[r:r + WINDOW_LENGTH]
            score += evaluate_window(window,piece)
            
    #Score positively sloped diagnols
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window,piece)
            
    #Score negatively sloped diagonals
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)
        
    return score

'''
def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0
    
def minimax(board, depth,alpha,beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal     = is_terminal_node(board)
    
    if depth==0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000)
            if winning_move(board, PLAYER_PIECE):
                return (None, -100000000000)
            else:
                return (None,0)
        else:
            return (None, score_position(board, AI_PIECE))
        
        
    if maximizingPlayer:
        value = -math.inf
        column  = random.choice(valid_locations)
        for col in valid_locations:
            row  = get_next_open_row(board,col)
            b_copy = board.copy()
            drop_piece(b_copy, row,col, AI_PIECE)
            new_score = minimax(b_copy, depth-1, alpha,beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha,value)
            if alpha>=beta:
                break
        return column, value
    
    else: #MinimizingPlayer
        val = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1,alpha, beta, True)[1]
            if new_score < val:
                value  = new_score
                column = col
            beta = min(beta, value)
            if alpha>=beta:
                break
        return column, value
'''

def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0

def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -10000000000000)
            else: # Game is over, no more valid moves
                return (None, 0)
        else: # Depth is zero
            return (None, score_position(board, AI_PIECE))
    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else: # Minimizing player
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value
    

def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board,col):
            valid_locations.append(col)
    return valid_locations

def pick_best_move(board,piece):
    
    valid_locations = get_valid_locations(board)
    
    best_score = -100000
    best_col = random.choice(valid_locations)
    
    for col in valid_locations:
        row = get_next_open_row(board,col)
        temp_board =board.copy()
        drop_piece(temp_board,row,col,piece)
        score = score_position(temp_board,piece)
        if score > best_score:
            best_score =score
            best_col = col
            
    return best_col
            

board = create_board()
#print_board(board)
game_over = False
#turn= PLAYER
turn = random.randint(PLAYER, AI)

pygame.init()

#squaresize = 100 #units(pixels)
width = COLUMN_COUNT*SQSIZE
height = (ROW_COUNT+1)*SQSIZE

size = (width,height)

screen = pygame.display.set_mode(size)
draw_board(board,screen)
pygame.display.update()

myfont = pygame.font.SysFont("monospace", 75)


while not game_over:
   
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()
            
        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, BLACK, (0,0, width,SQSIZE))
            posx = event.pos[0]
            if turn == PLAYER:
                pygame.draw.circle(screen,GREEN,(posx, int(SQSIZE/2)), RADIUS)
            
            #now we no longer should draw the circle above on black rect.
            #else:
             #  pygame.draw.circle(screen,YELLOW,(posx, int(SQSIZE/2)), RADIUS)
        pygame.display.update()
    
            
        
        
        if event.type == pygame.MOUSEBUTTONDOWN:
              #  print(event.pos)
             
            if turn == PLAYER :
                #Ask for Player1 Input
                #col = int(input("Player 1 Make your selection (0-6):"))
                posx = event.pos[0]
                col = int(math.floor(posx/SQSIZE))
            
                if is_valid_location(board,col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row,col,1)
        
                if(winning_move(board,1)):
                    #print("Player 1 wins")
                    #print("Player 2 learns")
                    #game_over=True
                    pygame.draw.rect(screen, BLACK, (0,0, width,SQSIZE))
                    label = myfont.render("Player 1 wins!!", 1, GREEN)
                    screen.blit(label, (40,10))
                    game_over = True
                    
                print_board(board)
                print("\n")
                draw_board(board,screen)
            
              
                turn = (turn+1)%2 
          
    if turn == AI and not game_over:
            #Ask player2 input
            # col = int(input("Player 2 Make your selection (0-6):")
            #col = random.randint(0,COLUMN_COUNT-1)
            #col = pick_best_move(board, AI_PIECE)
            
            #col, minimax_score = minimax(board, 3, True)
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
                
            if is_valid_location(board,col):
                #pygame.time.wait(500)
                row = get_next_open_row(board, col)
                drop_piece(board, row,col,2)
            
            if(winning_move(board,2)):
                    #print("Player 2 wins")
                    #print("Player 1 learns")
                    #game_over=True
                pygame.draw.rect(screen, BLACK, (0,0, width,SQSIZE))
                label = myfont.render("Player 2 wins!!", 2, YELLOW)
                screen.blit(label, (40,10))
                game_over = True
    
            print_board(board)
            print("\n")
            draw_board(board,screen)
            
            #The next if else to change the color of moving ball, once the piece has been dropped
  #if(not game_over):
    #   if(turn==0):
     #      pygame.draw.circle(screen,YELLOW,(posx, int(SQSIZE/2)), RADIUS)
      # else:
       #    pygame.draw.circle(screen,GREEN,(posx, int(SQSIZE/2)), RADIUS)'''
            
    #ygame.display.update()
            turn = (turn+1)%2 


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 1. 1. 2. 0. 0. 0.]]


